Create model

In [1]:
yellow_follow = 'C:/Users/caspe/Desktop/yellow/lib/'

import sys; sys.path.append(yellow_follow) 
import sqlite3
import pandas as pd
import ml_utils
import numpy as np
import os
from sqlalchemy import create_engine
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Conv3D, MaxPooling3D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

np.set_printoptions(suppress=True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
folder = "C:/Users/caspe/Desktop/Paper_6_Generative/"

# Load data
X_train = np.load(folder + "X_train.npy").transpose(0, 2, 3, 1)
y_train = np.load(folder + "y_train.npy")[..., np.newaxis]
X_test = np.load(folder + "X_test.npy").transpose(0, 2, 3, 1)
y_test = np.load(folder + "y_test.npy")[..., np.newaxis]

In [3]:
def lr_time_based_decay(epoch):
    return initial_learning_rate - (epoch * ((initial_learning_rate - end_learning_rate) / epochs))

def define_optimizer(learning_rate):
    return tfa.optimizers.Lookahead(
        Adam(
            learning_rate=learning_rate,
            name="Adam",
        )
    )

In [39]:
def define_model(shape, name):
    drop = 0.2
    acti = "elu"

    model_input = Input(shape=shape, name=name)
    model = Conv2D(128, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model_input)
    model = BatchNormalization()(model)
    model = Conv2D(128, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model_input)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = MaxPooling2D(pool_size=(2, 2), padding='same', strides=(2, 2))(model)

    model = Conv2D(96, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Conv2D(96, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    model = Conv2DTranspose(128, kernel_size=3, padding='same', strides=2, activation=acti, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)

    model = Conv2D(96, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Conv2D(64, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model)
    model = BatchNormalization()(model)
    model = Dropout(drop)(model)

    output = Conv2D(1, kernel_size=3, padding='same', activation=acti, kernel_initializer='he_uniform')(model)

    return Model(inputs=[model_input], outputs=output)

In [40]:
model = define_model(X_train.shape[1:], "Generative")
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generative (InputLayer)      [(None, 16, 16, 6)]       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 16, 16, 64)        3520      
_________________________________________________________________
batch_normalization_24 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 8, 8, 96)          55392     
_________________________________________________________________
batch_normalization_25 (Batc (None, 8, 8, 96)          384 

In [41]:
epochs = 10
initial_learning_rate = 0.001
end_learning_rate = 0.00001

model.compile(
    optimizer=define_optimizer(initial_learning_rate),
    loss="log_cosh",
    metrics=[
        'mse',
        'mae',
        'log_cosh',
    ]
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    verbose=1,
    batch_size=256,
    validation_split=0.2,
    callbacks=[
        LearningRateScheduler(lr_time_based_decay, verbose=1),
        EarlyStopping(
            monitor="val_loss",
            patience=9,
            min_delta=1,
            restore_best_weights=True,
        ),
    ]
)

Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
482/482 [==============================] - 28s 48ms/step - loss: 6.9413 - mse: 2769.5148 - mae: 7.0790 - log_cosh: 6.9413 - val_loss: 6.6261 - val_mse: 2666.8303 - val_mae: 6.6919 - val_log_cosh: 6.6261
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.00901.
482/482 [==============================] - 24s 49ms/step - loss: 6.4623 - mse: 2333.7146 - mae: 6.6114 - log_cosh: 6.4623 - val_loss: 6.6816 - val_mse: 2728.8379 - val_mae: 6.8174 - val_log_cosh: 6.6816
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00802.
482/482 [==============================] - 26s 54ms/step - loss: 6.2753 - mse: 2183.7292 - mae: 6.4401 - log_cosh: 6.2753 - val_loss: 5.9905 - val_mse: 2165.7642 - val_mae: 6.1060 - val_log_cosh: 5.9905
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning rate to 0.00703.
482/482 [==============================] - 28s 57ms/step - loss: 6

In [42]:
print(model.evaluate(X_test, y_test))

2464/2464 [==============================] - 13s 5ms/step - loss: 5.5705 - mse: 1999.6456 - mae: 5.6352 - log_cosh: 5.5705
[5.570501327514648, 1999.6456298828125, 5.635154724121094, 5.570501327514648]


In [ ]:
pred = model.predict(X_test)

In [ ]:
gimg = 2001
import matplotlib.pyplot as plt

def to_8bit(arr):
    red = (arr * 255 / y_test.max()).astype('uint8')
    return red[:, :, 0]

In [ ]:
plt.imshow(to_8bit(pred[gimg]), interpolation='none')
plt.show()

In [ ]:
plt.imshow(to_8bit(y_test[gimg]), interpolation='none')
plt.show()